# Exam on Convolutional Neural Networks (CNN)

Welcome to the Convolutional Neural Networks (CNN) practical exam. In this exam, you will work on an image classification task to predict the type of traffic sign. You are provided with a dataset of traffic sign images, and your task is to build, train, and evaluate a CNN model.

---

## Dataset Overview
### **Dataset:**
* Just run the command under the `Load Data` section to get the data downloaded and unzipped or you can access it [here](https://drive.google.com/file/d/1HwMV-Lt_sWoxc5v6igmTxTwomS3DR6cQ/view?usp=sharing)
### **Dataset Name:** Traffic Signs

### **Description:**  
The dataset contains images of various German traffic signs labeled for classification purposes. Each image belongs to one of the 43 classes, representing different types of traffic signs.

### **Labels:**
```python
classes = {
    0:  'Speed limit (20km/h)',
    1:  'Speed limit (30km/h)',
    2:  'Speed limit (50km/h)',
    3:  'Speed limit (60km/h)',
    4:  'Speed limit (70km/h)',
    5:  'Speed limit (80km/h)',
    6:  'End of speed limit (80km/h)',
    7:  'Speed limit (100km/h)',
    8:  'Speed limit (120km/h)',
    9:  'No passing',
    10: 'No passing veh over 3.5 tons',
    11: 'Right-of-way at intersection',
    12: 'Priority road',
    13: 'Yield',
    14: 'Stop',
    15: 'No vehicles',
    16: 'Veh > 3.5 tons prohibited',
    17: 'No entry',
    18: 'General caution',
    19: 'Dangerous curve left',
    20: 'Dangerous curve right',
    21: 'Double curve',
    22: 'Bumpy road',
    23: 'Slippery road',
    24: 'Road narrows on the right',
    25: 'Road work',
    26: 'Traffic signals',
    27: 'Pedestrians',
    28: 'Children crossing',
    29: 'Bicycles crossing',
    30: 'Beware of ice/snow',
    31: 'Wild animals crossing',
    32: 'End speed + passing limits',
    33: 'Turn right ahead',
    34: 'Turn left ahead',
    35: 'Ahead only',
    36: 'Go straight or right',
    37: 'Go straight or left',
    38: 'Keep right',
    39: 'Keep left',
    40: 'Roundabout mandatory',
    41: 'End of no passing',
    42: 'End no passing veh > 3.5 tons'
}
```



## Load Data
Run the following command to get the data and unzip it, alternatively you can access the data [here](https://drive.google.com/file/d/1HwMV-Lt_sWoxc5v6igmTxTwomS3DR6cQ/view?usp=sharing).

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
from sklearn.model_selection import train_test_split
from warnings import filters
import os
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

In [ ]:
!unzip '/content/drive/MyDrive/T5/week5/Traffic_Signs.zip'

## Import Libraries

In [12]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

In [13]:
df=pd.read_csv('/content/Traffic_Signs/Dataset_MetaData.csv')

In [15]:
df['labels']=df['Path'].apply(lambda x: x [8:10].replace('/',''))

In [16]:
df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,labels
0,27,26,5,5,22,20,20,Dataset/20/00020_00000_00000.png,20
1,28,27,5,6,23,22,20,Dataset/20/00020_00000_00001.png,20
2,29,26,6,5,24,21,20,Dataset/20/00020_00000_00002.png,20
3,28,27,5,6,23,22,20,Dataset/20/00020_00000_00003.png,20
4,28,26,5,5,23,21,20,Dataset/20/00020_00000_00004.png,20


## Data Preprocessing
In this section, preprocess the dataset by:
- Loading the images from the file paths.
- Resizing the images to a consistent size.
- Normalizing pixel values.

Add more if needed!

In [46]:
image_size = (180, 180)
batch_size = 128

data_augmentation_layers = [
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomBrightness(factor=0.2)
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

## Data Splitting
In this section, we will split our dataset into three parts:

* Training set (70%).
* Validation set (15%).
* Test set (15%).

In [47]:
train_ds = keras.utils.image_dataset_from_directory(
    "/content/Traffic_Signs/Dataset",
    labels='inferred',
    label_mode='int',#many labels
    subset="training",
    validation_split=0.15,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)


val_ds = keras.utils.image_dataset_from_directory(
    "/content/Traffic_Signs/Dataset",
    labels='inferred',
    label_mode='int',
    subset="validation",
    validation_split=0.15,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

test_ds = keras.utils.image_dataset_from_directory(
    "/content/Traffic_Signs/Dataset",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False
)


Found 39209 files belonging to 43 classes.
Using 33328 files for training.
Found 39209 files belonging to 43 classes.
Using 5881 files for validation.
Found 39209 files belonging to 43 classes.


In [48]:
augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

## Building the CNN Model
In this section, define the architecture of the CNN model. The architecture may consist of:
- Convolutional layers with max-pooling
- Dropout layers
- Flatten layer
- Dense layers
- Output layer

Add and remove any of these as needed!

In [49]:
model_cnn=keras.Sequential([

    layers.Input(shape=(image_size[0],image_size[1],3)),#input layer

    #block 1
    layers.Conv2D(filters=32,kernel_size=3,padding='same'),
    layers.Conv2D(filters=32,kernel_size=3,padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),

    #block 2
    layers.Conv2D(filters=32,kernel_size=3,padding='same'),
    layers.Conv2D(filters=32,kernel_size=3,padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),

    #block 3
    layers.Conv2D(filters=32,kernel_size=3,padding='same'),
    layers.Conv2D(filters=32,kernel_size=3,padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),

    layers.Flatten(),

    layers.Dense(units=64,activation='relu'),
    layers.Dense(units=32,activation='relu'),
    layers.Dense(units=16,activation='relu'),

    layers.Dense(units=1),



])

In [50]:
model_cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 180, 180, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 180, 180, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 90, 90, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 90, 90, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 90, 90, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 45, 45, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 45, 45, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 45, 45, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 22, 22, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 15488)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │         991,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,041,057 (3.97 MB)

 Trainable params: 1,041,057 (3.97 MB)

 Non-trainable params: 0 (0.00 B)

## Training the Model
Train the CNN model using the training data and validate it on the validation set.

In [ ]:
epochs = 3

callbacks = [
    keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model_cnn.compile(
    optimizer='adam',
    loss='BinaryCrossentropy',
    metrics=['Accuracy'],
)
model_cnn.fit(
    augmented_train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/3
 51/261 ━━━━━━━━━━━━━━━━━━━━ 1:01:27 18s/step - Accuracy: 0.0403 - loss: -159.2461

## Evaluate the Model
Evaluate the performance of the model on the test set.

In [ ]:
model_cnn.evaluate(test_ds)

## Make Predictions
Use the trained model to make predictions on new or unseen traffic sign images.

if you need new, we prepared some data for you [here](https://drive.google.com/file/d/1S_vpQntND9839x8kJpegaEgtSIA4JxHO/view?usp=sharing), or you can simply run the following command to get the data and unzip it.

<small>Note: please note that the file contain MetaData to tell you what each image contains <b>THIS IS JUST FOR YOU TO MAKE SURE</b></smmall>

In [ ]:
#https://drive.google.com/file/d/1S_vpQntND9839x8kJpegaEgtSIA4JxHO/view?usp=sharing
!gdown --id 1S_vpQntND9839x8kJpegaEgtSIA4JxHO
!unzip Test_Dataset.zip

In [ ]:
img=keras.utils.load_img('/content/Traffic_Signs/Dataset/11/00011_00000_00000.png',target_size=image_size) #take random image
plt.imshow(img)

img_array=keras.utils.img_to_array(img)
img_array=keras.ops.expand_dims(img_array,0)

Prediction =model_cnn.prediction(img_array)
score=float(keras.ops.softmax(Prediction[0][0]))#couldnot fininsh

## Model Performance Visualization
Visualize performance metrics such as accuracy and loss over the epochs.

In [ ]:
plt.figure(figsize=(14,5))#take visual from task 1

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')

plt.show()

In [ ]:
plt.figure(figsize=(14,5))#take visual from task 1

plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.show()

## Save the Model
Save the trained CNN model for submission.

In [ ]:
model_ann.save('weekly_5_cnn.keras')

## Project Questions:

1. **Data Preprocessing**: Explain why you chose your specific data preprocessing techniques (e.g., resizing images, normalization, data augmentation). How do these preprocessing steps improve the performance of your CNN model?
2. **Model Architecture**: Describe the architecture of your CNN model (e.g., number of convolutional layers, kernel sizes, pooling layers). Why did you choose this structure, and how do you expect each layer to contribute to feature extraction?
3. **Activation Functions**: Justify your choice of activation functions. How do they influence the training and output of your CNN?
4. **Training Process**: Discuss your choice of batch size, number of epochs, and optimizer. How did these decisions impact the training process and the convergence of the model?
5. **Loss Function and Metrics**: Explain why you chose the specific loss function and evaluation metrics for this classification task. How do they align with the goal of correctly classifying traffic signs?
6. **Regularization Techniques**: If you used regularization methods like dropout or batch normalization, explain why you implemented them and how they helped prevent overfitting in your model.
7. **Model Evaluation**: Justify the method you used to evaluate your model's performance on the test set. Why did you select these evaluation techniques, and what insights did they provide about your model's accuracy and generalization ability?
8. **Model Visualization**: Explain the significance of the performance visualizations (e.g., accuracy and loss curves). What do they tell you about your model's training process and its ability to generalize?
9. **Overfitting and Underfitting**: Analyze whether the model encountered any overfitting or underfitting during training. What strategies could you implement to mitigate these issues?

### Answer Here: